In [2]:
import pandas as pd
from deltalake import DeltaTable

DELTA_PATH = "../data/raw_reddit_posts_comments"
delta_table = DeltaTable(DELTA_PATH)

df = delta_table.to_pandas()
df

,post_title,post_score,post_id,post_url,post_name,post_author_name,level,comment_id,comment_name,comment_upvotes,comment_downvotes,comment_created_at,comment_edited_at,comment_author_name,comment_author_fullname,comment_parent_id,comment_body
0,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,comment,nnakzjc,t1_nnakzjc,986,0,2025-11-06 03:44:31,2025-11-06 11:50:30,Cluelessbigirl,t2_duy9z7kw,t3_1opcev8,"2029 may sound like a long wait, but I’m kinda..."
1,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnaold6,t1_nnaold6,205,0,2025-11-06 04:01:42,1970-01-01 08:00:00,Hero_of_the_toons,t2_64w7o0ao,t1_nnakzjc,Wouldn’t want a Kpdh equivalent of Sonic 06 if...
2,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnb3wf6,t1_nnb3wf6,79,0,2025-11-06 05:15:41,1970-01-01 08:00:00,CelineShotFirst,t2_1rqf9pzis9,t1_nnakzjc,The original film took 7 years from pitch to r...
3,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnaomsc,t1_nnaomsc,50,0,2025-11-06 04:01:54,1970-01-01 08:00:00,AlternativeJeweler6,t2_4gxabdys,t1_nnakzjc,"If nobody in production has to crunch, it’ll b..."
4,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnasea2,t1_nnasea2,15,0,2025-11-06 04:20:14,1970-01-01 08:00:00,TY-KLR,t2_22bpntxz,t1_nnakzjc,Exactly this! I’ve been saying the same thing ...
5,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnbb3r2,t1_nnbb3r2,10,0,2025-11-06 05:49:39,1970-01-01 08:00:00,nottytom,t2_8gqqxobe,t1_nnakzjc,"totally with you, I was worried they rush them..."
6,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnbvjil,t1_nnbvjil,9,0,2025-11-06 07:38:44,2025-11-06 07:43:14,Brief-Promise-8882,t2_20dorjkpw0,t1_nnakzjc,I'm just praying they write something actually...
7,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnavof0,t1_nnavof0,7,0,2025-11-06 04:36:12,1970-01-01 08:00:00,Cynicbats,t2_9ih3b,t1_nnakzjc,I also expect Netflix to heave resources at th...
8,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nnazugt,t1_nnazugt,3,0,2025-11-06 04:56:31,1970-01-01 08:00:00,Dying_light2fan,t2_anx7nmbsw,t1_nnakzjc,That’s like a very long time away
9,‘KPop Demon Hunters 2’ Aims for 2029 Release o...,2121,1opcev8,https://variety.com/2025/film/news/kpop-demon-...,t3_1opcev8,DemiFiendRSA,reply,nndc9we,t1_nndc9we,3,0,2025-11-06 13:09:53,1970-01-01 08:00:00,wolfej4,t2_5skew,t1_nnakzjc,I can’t say I’m surprised. There was a 5 year ...


https://delta-docs-incubator.netlify.app/delta-update/#slowly-changing-data-scd-type-2-operation-into-delta-tables

Upsert into a table using merge
You can upsert data from a source table, view, or DataFrame into a target Delta table by using the MERGE SQL operation. Delta Lake supports inserts, updates and deletes in MERGE, and it supports extended syntax beyond the SQL standards to facilitate advanced use cases.

Suppose you have a source table named people10mupdates or a source path at /tmp/delta/people-10m-updates that contains new data for a target table named people10m or a target path at /tmp/delta/people-10m. Some of these new records may already be present in the target data. To merge the new data, you want to update rows where the person’s id is already present and insert the new rows where no matching id is present. You can run the following:

```
MERGE INTO people10m
USING people10mupdates
ON people10m.id = people10mupdates.id
WHEN MATCHED THEN
  UPDATE SET
    id = people10mupdates.id,
    firstName = people10mupdates.firstName,
    middleName = people10mupdates.middleName,
    lastName = people10mupdates.lastName,
    gender = people10mupdates.gender,
    birthDate = people10mupdates.birthDate,
    ssn = people10mupdates.ssn,
    salary = people10mupdates.salary
WHEN NOT MATCHED
  THEN INSERT (
    id,
    firstName,
    middleName,
    lastName,
    gender,
    birthDate,
    ssn,
    salary
  )
  VALUES (
    people10mupdates.id,
    people10mupdates.firstName,
    people10mupdates.middleName,
    people10mupdates.lastName,
    people10mupdates.gender,
    people10mupdates.birthDate,
    people10mupdates.ssn,
    people10mupdates.salary
  )
  ```